### New install

In [1]:
!pip install --upgrade jupyterlab
!pip install --upgrade jupyter
!pip install ipywidgets

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     |████████████████████████████████| 9.2 MB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 377 kB 120.6 MB/s eta 0:00:01
     |████████████████████████████████| 65 kB 103.7 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 77.2 MB/s eta 0:00:01
     |████████████████████████████████| 427 kB 106.0 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 91.9 MB/s  eta 0:00:01
     |████████████████████████████████| 256 kB 155.1 MB/s eta 0:00:01
     |████████████████████████████████| 1.1 MB 102.5 MB/s eta 0:00:01
     |████████████████████████████████| 84 kB 104.4 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 115.9 MB/s eta 0:00:01
     |████████████████████████████████| 57 kB 109.9 MB/s eta 0:00:01
     |████████████████████████████████| 105 kB 130.1 MB/s eta 0:00:01
     |████████████████████████████████| 83 kB 103.5 MB/s eta 0:00:01
     |█████████████████████

## Imports

In [1]:
import logging
import sys
sys.path.append('../../src')
import timm
import torch
import torch.nn as nn
import torchaudio.transforms as AT
import torchvision.transforms as VT
from nnAudio import features

import config

Created directory: /dli/data/plots
Created directory: /dli/data/features/MED
Created directory: /dli/models


In [2]:
def quant_prep(model_ft):
    model_ft[0].qconfig = torch.quantization.default_qat_qconfig  # Use default QAT configuration
# Step 3
    model_ft = torch.quantization.prepare_qat(model_ft, inplace=True)
    for param in model_ft.parameters():
        param.requires_grad = True
    return(model_ft)

In [3]:
import torchvision.models.quantization as models
model = models.resnet18(pretrained=True, progress=True, quantize=False)
num_ftrs = model.fc.in_features
model.train()
model.fuse_model()


/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [4]:
from torch import nn

def create_combined_model(model_fe):
    # Step 1. Isolate the feature extractor.
    model_fe_features = nn.Sequential(
    model_fe.quant,  # Quantize the input
    model_fe.conv1,
    model_fe.bn1,
    model_fe.relu,
    model_fe.maxpool,
    model_fe.layer1,
    model_fe.layer2,
    model_fe.layer3,
    model_fe.layer4,
    model_fe.avgpool,
    model_fe.dequant,  # Dequantize the output
  )

    # Step 2. Create a new "head"
    new_head = nn.Sequential(
    nn.Dropout(p=0.5),
    nn.Linear(num_ftrs, 2),
   )

  # Step 3. Combine, and don't forget the quant stubs.
    new_model = nn.Sequential(
    model_fe_features,
    nn.Flatten(1),
    new_head,)
    
    model_ft  = quant_prep(new_model)
    
    return model_ft

In [7]:


def print_size_of_model(model):
    torch.save(model.state_dict(), "temp.p")
    print('Size (MB):', os.path.getsize("temp.p")/1e6)
    os.remove('temp.p')

In [12]:
def load_model(model_file,model = create_combined_model(model)):
    model =model 
    state_dict = torch.load(model_file)
    model.load_state_dict(state_dict)
    model.to('cpu')
    return model

/opt/conda/lib/python3.8/site-packages/torch/ao/quantization/observer.py:214: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(


## Load the base fp-32 model

In [10]:
!pwd

/dli/task


In [13]:
import os
filename = os.path.join("models","model_med0_2023_11_08_22_44_10.pth")

#state_dict = torch.load(filename)
model_ft_tuned  = load_model(filename)
model_ft_tuned.train()




Sequential(
  (0): Sequential(
    (0): QuantStub(
      (activation_post_process): FakeQuantize(
        fake_quant_enabled=tensor([1], dtype=torch.uint8), observer_enabled=tensor([1], dtype=torch.uint8), quant_min=0, quant_max=127, dtype=torch.quint8, qscheme=torch.per_tensor_affine, ch_axis=-1, scale=tensor([0.7838]), zero_point=tensor([0], dtype=torch.int32)
        (activation_post_process): MovingAverageMinMaxObserver(min_val=-0.11287054419517517, max_val=99.4275131225586)
      )
    )
    (1): ConvBnReLU2d(
      3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (weight_fake_quant): FakeQuantize(
        fake_quant_enabled=tensor([1], dtype=torch.uint8), observer_enabled=tensor([1], dtype=torch.uint8), quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric, ch_axis=-1, scale=tensor([0.0047]), zero_point=tensor([0], dtype=torch.int32)

In [11]:
#let's try to save the float model


In [12]:
def save_torchscript_model(model, model_dir, model_filename):

    if not os.path.exists(model_dir):
        os.makedirs(model_dir)
    model_filepath = os.path.join(model_dir, model_filename)
    torch.jit.save(torch.jit.script(model), model_filepath)

## Quantization starts here

In [14]:
from torch.quantization import convert


In [15]:
model_ft_tuned.cpu()

Sequential(
  (0): Sequential(
    (0): QuantStub(
      (activation_post_process): FakeQuantize(
        fake_quant_enabled=tensor([1], dtype=torch.uint8), observer_enabled=tensor([1], dtype=torch.uint8), quant_min=0, quant_max=127, dtype=torch.quint8, qscheme=torch.per_tensor_affine, ch_axis=-1, scale=tensor([0.7838]), zero_point=tensor([0], dtype=torch.int32)
        (activation_post_process): MovingAverageMinMaxObserver(min_val=-0.11287054419517517, max_val=99.4275131225586)
      )
    )
    (1): ConvBnReLU2d(
      3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (weight_fake_quant): FakeQuantize(
        fake_quant_enabled=tensor([1], dtype=torch.uint8), observer_enabled=tensor([1], dtype=torch.uint8), quant_min=-128, quant_max=127, dtype=torch.qint8, qscheme=torch.per_tensor_symmetric, ch_axis=-1, scale=tensor([0.0047]), zero_point=tensor([0], dtype=torch.int32)

In [16]:
model_quantized_and_trained = convert(model_ft_tuned, inplace=False)

In [17]:
torch.jit.script(model_quantized_and_trained ,"resnet_ft.pt")

/opt/conda/lib/python3.8/site-packages/torch/jit/_script.py:1277: UserWarning: `optimize` is deprecated and has no effect. Use `with torch.jit.optimized_execution() instead
  warnings.warn(


RecursiveScriptModule(
  original_name=Sequential
  (0): RecursiveScriptModule(
    original_name=Sequential
    (0): RecursiveScriptModule(original_name=Quantize)
    (1): RecursiveScriptModule(original_name=ConvReLU2d)
    (2): RecursiveScriptModule(original_name=Identity)
    (3): RecursiveScriptModule(original_name=Identity)
    (4): RecursiveScriptModule(original_name=MaxPool2d)
    (5): RecursiveScriptModule(
      original_name=Sequential
      (0): RecursiveScriptModule(
        original_name=QuantizableBasicBlock
        (conv1): RecursiveScriptModule(original_name=ConvReLU2d)
        (bn1): RecursiveScriptModule(original_name=Identity)
        (relu): RecursiveScriptModule(original_name=Identity)
        (conv2): RecursiveScriptModule(original_name=Conv2d)
        (bn2): RecursiveScriptModule(original_name=Identity)
        (add_relu): RecursiveScriptModule(
          original_name=QFunctional
          (activation_post_process): RecursiveScriptModule(original_name=Identity)


In [9]:
#model_int8_new_t_no_qnt_layer = torch.quantization.convert(model_fp32_prepared.to('cpu').eval())

/opt/conda/lib/python3.8/site-packages/torch/ao/quantization/utils.py:317: UserWarning: must run observer before calling calculate_qparams. Returning default values.
  warnings.warn(
